In [1]:
%load_ext autoreload
%autoreload 2
import torch
import random

from sae.sparse_autoencoder import load_saved_sae
from sae.metrics import model_store_from_sae
from unlearning.metrics import convert_wmdp_data_to_prompt
from unlearning.tool import UnlearningConfig, SAEUnlearningTool, MCQ_ActivationStoreAnalysis

from huggingface_hub import hf_hub_download
from datasets import load_dataset
import numpy as np
import pandas as pd
import itertools
from transformer_lens import utils

from jaxtyping import Float
from torch import Tensor

import plotly.express as px


In [2]:
# resid pre 9
REPO_ID = "eoinf/unlearning_saes"
FILENAME = "jolly-dream-40/sparse_autoencoder_gemma-2b-it_blocks.9.hook_resid_pre_s16384_127995904.pt"


filename = hf_hub_download(repo_id=REPO_ID, filename=FILENAME)
sae = load_saved_sae(filename)

model = model_store_from_sae(sae)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model gemma-2b-it into HookedTransformer
Moving model to device:  cuda


In [3]:
# read 172 questions that the model can answer correctly in any permutation
filename = '../data/wmdp-bio_gemma_2b_it_correct.csv'
correct_question_ids = np.genfromtxt(filename)


# read 133 questions that the model can answer correctly in any permutation but will get it wrong if
# without the instruction prompt and the question prompt
filename = '../data/wmdp-bio_gemma_2b_it_correct_not_correct_wo_question_prompt.csv'
correct_question_id_not_correct_wo_question_prompt = np.genfromtxt(filename)


In [4]:
dataset = load_dataset("cais/wmdp", "wmdp-bio")
permute_choices = None # (2, 1, 3, 0)
prompts = [convert_wmdp_data_to_prompt(dataset['test'][i]['question'], dataset['test'][i]['choices'], prompt_format=None, permute_choices=permute_choices) for i in range(len(dataset['test'])) if i in correct_question_id_not_correct_wo_question_prompt]

In [5]:
# setup unlearning tool, need about 3 minutes to run this cell

filename = "../data/wmdp-bio_gemma_2b_it_correct.csv"
correct_question_ids = np.genfromtxt(filename)

dataset_args = {
    'question_subset': correct_question_ids,
}

sae.cfg.n_batches_in_store_buffer = 86
unlearn_activation_store = MCQ_ActivationStoreAnalysis(sae.cfg, model, dataset_args=dataset_args)
unlearning_metric = 'wmdp-bio_gemma_2b_it_correct'


unlearn_cfg = UnlearningConfig(unlearn_activation_store=unlearn_activation_store, unlearning_metric=unlearning_metric)
ul_tool = SAEUnlearningTool(unlearn_cfg)
ul_tool.setup()
# ul_tool.get_metrics_with_text()

dataloader


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model gemma-2b-it into HookedTransformer
Moving model to device:  cuda
buffer
dataloader


In [6]:
# load good feature list
import pickle
with open('./unlearning_output/good_features_list_v1.pkl', 'rb') as f:
    good_features_list = pickle.load(f)

good_features_list

{70: [4802],
 89: [2993],
 158: [11019, 4802],
 172: [2993, 12273, 4802],
 190: [2993, 6958],
 192: [4342, 6325, 2993],
 200: [9280],
 203: [2993, 4802, 1307],
 216: [3652, 4802],
 217: [1523, 2993],
 218: [],
 243: [2993, 4802],
 265: [2993],
 267: [4802],
 314: [4802, 6273],
 324: [7983],
 345: [2993, 4802, 15848],
 348: [14315],
 353: [2993, 4802, 6958, 3652],
 354: [2993, 4802],
 357: [4802, 2993],
 359: [4802],
 360: [4802, 2993],
 362: [2993, 4802],
 367: [],
 373: [2993, 12663, 4802],
 375: [2993, 6531, 4802],
 376: [4802],
 377: [4802],
 378: [4291],
 384: [2993],
 405: [4291, 4802, 5691],
 447: [3652],
 452: [4802, 2993],
 474: [15858],
 479: [2993],
 482: [],
 494: [10355, 2993, 9391],
 513: [1746, 4802],
 534: [6273, 4802],
 542: [2993, 12289, 12663],
 584: [2993],
 600: [15858, 12289],
 612: [2993, 4802],
 617: [2993, 4802],
 626: [2993],
 630: [3652],
 636: [2993],
 645: [2993, 1611, 10051, 12289],
 649: [2993, 4802],
 650: [5749, 16186, 4802, 4451, 5861],
 652: [2993, 143

In [7]:
len(good_features_list)

133

In [8]:
good_features = []
for q, feature_list in good_features_list.items():
    good_features.extend(feature_list)
    
good_features = list(set(good_features))
print(len(good_features))

41


In [9]:
good_features

[6273,
 12289,
 6531,
 5001,
 11019,
 15755,
 3728,
 1307,
 6958,
 7983,
 9391,
 2993,
 2222,
 4654,
 14388,
 6325,
 16186,
 5691,
 9280,
 4802,
 4291,
 3652,
 10051,
 1611,
 1746,
 7122,
 8412,
 4451,
 14819,
 5861,
 15848,
 14315,
 12273,
 15858,
 1523,
 10355,
 5749,
 4342,
 12663,
 8946,
 12287]

In [46]:
filtered_good_features = [12663, 4342, 5749, 10355, 1523, 15858, 12273, 14315, 4451, 1611, 10051, 16186, 7983, 6958, 1307, 11019, 6531, 12289]

In [47]:
len(filtered_good_features)

18

In [48]:
features_to_ablate = filtered_good_features # [12273, 11237, 7956, 4451, 2002]
multiplier = 20
all_permutations = list(itertools.permutations([0, 1, 2, 3]))

ablate_params = {
    'features_to_ablate': features_to_ablate,
    'multiplier': multiplier,
    'intervention_method': 'scale_feature_activation',
    'permutations': all_permutations
}

metrics = ul_tool.calculate_metrics(**ablate_params)


# calc control metric and loss
control_metrics = ul_tool.calculate_control_metrics(random_select_one=False, **ablate_params)

loss_added = ul_tool.compute_loss_added(n_batch=30, **ablate_params)

100%|██████████| 30/30 [00:43<00:00,  1.45s/it]


In [50]:
print(f"Metrics: {metrics['modified_metrics']['mean_correct']}")
print(f"Control metrics: {control_metrics['mean_correct']}")
print(f"Loss added: {loss_added}")

Metrics: 0.32776162028312683
Control metrics: 0.8776881694793701
Loss added: 0.06919546127319336


In [11]:
feature_performance = {}

for feature in good_features:
    ablate_params = {
        'features_to_ablate': [feature],
        'multiplier': 20,
        'intervention_method': 'scale_feature_activation',
        'permutations': [(0,1,2,3)],
        'verbose': False
    }

    metrics = ul_tool.calculate_metrics(**ablate_params)

    # calc control metric and loss
    control_metrics = ul_tool.calculate_control_metrics(random_select_one=False, **ablate_params)

    loss_added = ul_tool.compute_loss_added(n_batch=10, **ablate_params)

    print(f"Feature: {feature}")
    print(f"Metrics: {metrics['modified_metrics']['mean_correct']}")
    print(f"Control metrics: {control_metrics['mean_correct']}")
    print(f"Loss added: {loss_added}")
    print("\n\n")
    
    feature_performance[feature] = {
        'metrics': metrics,
        'control_metrics': control_metrics,
        'loss_added': loss_added
    }
    

100%|██████████| 10/10 [00:14<00:00,  1.45s/it]


Feature: 6273
Metrics: 0.29651162028312683
Control metrics: 0.30000001192092896
Loss added: 0.05593559741973877





100%|██████████| 10/10 [00:14<00:00,  1.42s/it]


Feature: 12289
Metrics: 0.8372092843055725
Control metrics: 1.0
Loss added: -1.2254714965820312e-05





100%|██████████| 10/10 [00:14<00:00,  1.44s/it]


...

Feature: 12663
Metrics: 0.8895348906517029
Control metrics: 0.9333333969116211
Loss added: 0.0026502609252929688





100%|██████████| 10/10 [00:14<00:00,  1.45s/it]


Feature: 8946
Metrics: 1.0
Control metrics: 1.0
Loss added: 0.0007244825363159179





100%|██████████| 10/10 [00:14<00:00,  1.45s/it]

Feature: 12287
Metrics: 0.9767441749572754
Control metrics: 1.0
Loss added: 1.066833209991455





In [13]:
feature_performance

{6273: {'metrics': {'baseline_metrics': {'mean_correct': 1.0,
    'total_correct': 168,
    'is_correct': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
    'output_probs': array([[1.84581804e-05, 4.15313451e-07, 1.17745221e-05, 9.98588264e-01],
           [7.6647076

In [16]:
# feature_performance[feature] = {
#     'metrics': metrics,
#     'control_metrics': control_metrics,
#     'loss_added': loss_added
# }

filtered_good_features = []

for feature, performance in feature_performance.items():
    loss_added = performance['loss_added']
    control_metrics = performance['control_metrics']
    metrics = performance['metrics']
    
    if loss_added > 0.01:
        continue
    
    if control_metrics['mean_correct'] < 1:
        continue
    
    print(f"Feature: {feature}")
    print(f"Metrics: {metrics['modified_metrics']['mean_correct']}")
    print(f"Control metrics: {control_metrics['mean_correct']}")
    print(f"Loss added: {loss_added}")
    print("\n\n")
    
    
    filtered_good_features.append(feature)
    

Feature: 12289
Metrics: 0.8372092843055725
Control metrics: 1.0
Loss added: -1.2254714965820312e-05



Feature: 6531
Metrics: 0.9534883499145508
Control metrics: 1.0
Loss added: 0.002674698829650879



Feature: 5001
Metrics: 0.9825581312179565
Control metrics: 1.0
Loss added: -1.0728836059570312e-06



Feature: 11019
Metrics: 0.9302325248718262
Control metrics: 1.0
Loss added: -0.0016399621963500977



Feature: 15755
Metrics: 0.9709302186965942
Control metrics: 1.0
Loss added: -0.00028119087219238283



Feature: 6958
Metrics: 0.9360464811325073
Control metrics: 1.0
Loss added: 1.6570091247558594e-05



Feature: 7983
Metrics: 0.9883720874786377
Control metrics: 1.0
Loss added: -0.00010442733764648438



Feature: 2222
Metrics: 1.0
Control metrics: 1.0
Loss added: 0.0020005226135253905



Feature: 4654
Metrics: 0.9709302186965942
Control metrics: 1.0
Loss added: 0.00014357566833496095



Feature: 16186
Metrics: 0.9941860437393188
Control metrics: 1.0
Loss added: -0.0006730318069458008





In [18]:
len(filtered_good_features)
# [12289, 6531, 5001, 11019, 15755, 6958, 7983, 2222, 4654, 16186, 10051, 1746, 7122, 8412, 14315, 12273, 15858, 10355, 5749, 4342, 8946]

21

In [20]:
print(filtered_good_features)

[12289, 6531, 5001, 11019, 15755, 6958, 7983, 2222, 4654, 16186, 10051, 1746, 7122, 8412, 14315, 12273, 15858, 10355, 5749, 4342, 8946]


In [57]:
# get the indices of the correct_question_id_not_correct_wo_question_prompt questions in
# correct_question_ids
focus_questions = [i for i, q in enumerate(correct_question_ids) if q in correct_question_id_not_correct_wo_question_prompt]
not_focus_questions = [i for i in range(len(correct_question_ids)) if i not in focus_questions]
print(len(focus_questions), len(not_focus_questions))

acc_on_focus_questions = metrics['modified_metrics']['is_correct'].reshape(-1, 24)[focus_questions].mean(axis=-1)# .mean()
acc_on_not_focus_questions = metrics['modified_metrics']['is_correct'].reshape(-1, 24)[not_focus_questions].mean(axis=-1) #mean()

# print(f"Accuracy on 133 focus questions: {acc_on_focus_questions}")
# print(f"Accuracy on not focus questions: {acc_on_not_focus_questions}")

133 39


In [60]:
acc_on_not_focus_questions.mean()

0.4188034

In [59]:
acc_on_focus_questions.mean()

0.30106518

In [38]:
metrics['modified_metrics']['is_correct'].reshape(-1, 24).shape

(172, 24)

In [40]:
focus_questions_mask

array([[False,  True, False, ..., False, False, False],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True, False,  True, ...,  True,  True,  True],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [17]:
control_metrics

{'mean_correct': 0.25806450843811035,
 'total_correct': 192,
 'is_correct': array([0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0.,
        0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.,
        0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0.,
        0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0.,
        0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1.,
        0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.,
        0., 0

In [18]:
loss_added

7.372343182563782